In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
!which python

/media/mike/internal/virtualenv/py36/bin/python


In [3]:
import os

import numpy as np
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

from zoobot.active_learning import metrics
from zoobot.tests import TEST_FIGURE_DIR


In [4]:
samples = np.load('/home/mike/repos/zoobot/analysis/uncertainty/al-binomial/five_conv_fractions/samples.npy')
labels = np.load('/home/mike/repos/zoobot/analysis/uncertainty/al-binomial/five_conv_fractions/labels.npy')
model = metrics.Model(samples, labels, 'five_conv_fractions')

In [5]:
plt.hist(model.abs_error * 40)

(array([428., 307., 183.,  67.,  24.,   5.,   8.,   1.,   0.,   1.]),
 array([1.69157401e-02, 2.20737049e+00, 4.39782524e+00, 6.58827999e+00,
        8.77873474e+00, 1.09691895e+01, 1.31596442e+01, 1.53500990e+01,
        1.75405537e+01, 1.97310085e+01, 2.19214632e+01]),
 <a list of 10 Patch objects>)

In [6]:
plt.savefig('temp.png')

In [7]:
!pwd

/home/mike/repos/zoobot/zoobot/uncertainty


In [8]:
from zoobot.uncertainty import discrete_coverage

In [9]:
votes = np.around(labels*41)

In [10]:
votes.min(), votes.max()

(0.0, 39.0)

In [11]:
coverage_df = discrete_coverage.evaluate_discrete_coverage(votes, model.bin_probs)

In [12]:
coverage_df.head()

,actual_k,max_k,max_state_error,min_k,most_likely_k,observed,prediction
0,13.0,10,0,10,10,0.0,0.113723
1,13.0,11,1,9,10,0.0,0.331647
2,13.0,12,2,8,10,0.0,0.523501
3,13.0,13,3,7,10,1.0,0.679109
4,13.0,14,4,6,10,1.0,0.795854


In [13]:
len(coverage_df)

43008

In [14]:
reduced_df = discrete_coverage.reduce_coverage_df(coverage_df)

In [15]:
reduced_df.head(20)

,max_state_error,prediction,observed
0,0,0.123760,0.091797
1,1,0.343576,0.272461
2,2,0.530408,0.431641
3,3,0.679020,0.567383
4,4,0.789695,0.709961
5,5,0.867187,0.815430
6,6,0.918550,0.876953
7,7,0.951205,0.919922
8,8,0.971337,0.947266
9,9,0.983458,0.964844


In [26]:
coverage_df = discrete_coverage.evaluate_discrete_coverage(model.votes, model.bin_probs)
coverage_df_single = discrete_coverage.evaluate_discrete_coverage(model.votes, np.expand_dims(model.bin_probs[:, 0, :], axis=1))

In [31]:
sns.set_context('talk')
fig, (ax0, ax1) = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(8, 6))
discrete_coverage.plot_coverage_df(coverage_df_single, ax0)  
discrete_coverage.plot_coverage_df(coverage_df, ax1)
ax0.set_title('Samples = 1')
ax1.set_title('Samples = 30')
ax0.set_xlim([0, 20])
ax1.set_xlim([0, 20])
fig.tight_layout()
fig.savefig('discrete_coverage_comparison.png')

In [20]:
# shuffled_votes = model.votes.copy()
# np.random.shuffle(shuffled_votes)  # inplace
# coverage_df_random = discrete_coverage.evaluate_discrete_coverage(shuffled_votes, model.bin_probs)
# discrete_coverage.plot_coverage_df(coverage_df_random, 'discrete_coverage_random.png')

AttributeError: 'str' object has no attribute 'plot'